In [1]:
from ovcg import *
from fesol import *
from pyccx import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor
from mapper import generate_mapper

In [2]:
k=5
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

ccx_logoff = True
if ccx_logoff:
    import logging
    logger = logging.getLogger('PYCCX')
    logger.setLevel(logging.CRITICAL)

use_relax = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSN_avg%i.pvd' % k)
fnodes = solverF.get_coordinates_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)

In [4]:
# solid solver
jobname = 'solid/solidN'
output = 'asc'
prob = Problem(jobname=jobname, output=output)
msolver = prob.get_solver()
msolver.initailize()
# since we know there is one step
# we stop b4 it
msolver.solve(stopb4step=1)
nlgeom = msolver.get_nlgeomsolver()
nlgeom.initialize()

# initialize our interface set
iset = nlgeom.get_set(
    set_name='Interface',
    set_type='sfelem',
    is_surface=True
)
nset = nlgeom.get_set(
    set_name='Inodes',
    set_type='node',
    is_surface=True
)

# retrieve geometry data
snodes = nset.coordinates()
scents = iset.extract_face_centers()

# create cht adapter
solverS = CHTAdapter(nlgeom=nlgeom)
solverS.add_interface(set_acc=nset, is_settable=False, itype=CHTAdapter.TEMP)
solverS.add_interface(set_acc=iset, is_settable=True, itype=CHTAdapter.DFLUX)

# now we are ready

INFO - mastersolver: CCX jobname: solid/solidN, output format: asc.
INFO - mastersolver: Calling LIBCCX_solverInit...
INFO - mastersolver: LIBCCX_solverInit done.
INFO - mastersolver: Solving the problem until step 1.
INFO - mastersolver: And, done!
INFO - mastersolver: Generating the nonlinear geometrical solver...
INFO - nlgeomsolver: Calling LIBCCX_stepInit...
INFO - nlgeomsolver: LIBCCX_stepInit done.
INFO - genericaccessor: Set name: Interface, set id: 4, set type: T.
INFO - genericaccessor: Updating or getting topology data of set Interface.
INFO - genericaccessor: Set size: 264.
INFO - genericaccessor: Updating or getting connectivity...
INFO - genericaccessor: Finish getting topology data for set Interface.
INFO - genericaccessor: Set name: Inodes, set id: 3, set type: N.
INFO - genericaccessor: Updating or getting topology data of set Inodes.
INFO - genericaccessor: Set size: 171.
INFO - genericaccessor: Finish getting topology data for set Inodes.
INFO - chtadapter: Setting i

In [5]:
Tf = 'Tf'
Ts = 'Ts'
Ff = 'Ff'
Fs = 'Fs'
Ff2Fs = {Ff:Fs}
Ts2Tf = {Ts:Tf}
f2s_mapper, s2f_mapper = generate_mapper(fnodes, [scents, snodes], Ff2Fs, Ts2Tf,r1=0.05,r2=0.2)

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=fnodes.shape[0], value=0.0)
fluxS = InterfaceData(size=scents.shape[0], value=0.0)
tempF = InterfaceData(size=fnodes.shape[0], value=300.0)
tempS = InterfaceData(size=snodes.shape[0], value=310.0)
thetaT = InterfaceData(size=fnodes.shape[0], value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
t = 0.0
Nmax = 500
avg_pc_iterations = 0.0
flog = open('FDSNavg%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Neumann setting, study case: %i\n' % k)
flog.close()
flog = open('FDSNavg%i.log' % k, mode='a')

tempf_prev = tempF.curr.copy()
fluxs_prev = fluxS.curr.copy()

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup_state()
    
    # get previous time step value
    tempf_prev[:] = tempF.curr
    fluxs_prev[:] = fluxS.curr
    
    # mapping
    s2f_mapper.put(Ts, tempS.curr)
    s2f_mapper.apply(Ts, Tf)
    tempF.curr[:] = s2f_mapper.get(Tf)
    
    solverF.put_temperature_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=tempF.curr
    )
    
    while True:
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface flux
        fluxF.curr[:] = solverF.get_heat_flux_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        f2s_mapper.put(Ff, fluxF.curr)
        f2s_mapper.apply(Ff, Fs)
        fluxS.curr[:] = f2s_mapper.get(Fs)
        # update solid flux interface by taking the average
        solverS['Interface', SET].set_dfluxes(0.5*(fluxS.curr+fluxs_prev))
        
        # advance solid
        solverS.adjust_timesize(dt)
        solverS.increment()
        # retrieve solid interface temperature
        tempS.curr[:] = solverS['Inodes', GET].get_temperatures()
        
        # mapping
        s2f_mapper.put(Ts, tempS.curr)
        s2f_mapper.apply(Ts, Tf)
        tempF.curr[:] = s2f_mapper.get(Tf)
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverS.finish_increment()
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF)
                under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=0.5*(tempF.curr+tempf_prev)
            )
            solverF.restore_previous_solutions()
            solverS.restore_state()
            solverS.finish_increment()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        fluid_ofile.write(solverF, t)
flog.close()

nlgeom.finalize()
msolver.solve(skipsolve=True)
msolver.finalize()

INFO - chtadapter: Backup current state...
INFO - chtadapter: Backup finished.
INFO - chtapp: Assign dfluxes...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting temperature...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtapp: Assign dfluxes...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting temperature...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a conver

step=0, pc_iterations=9.


INFO - chtadapter: Backup current state...
INFO - chtadapter: Backup finished.
INFO - chtapp: Assign dfluxes...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting temperature...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtapp: Assign dfluxes...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting temperature...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a conver

step=1, pc_iterations=4.


INFO - chtapp: Assign dfluxes...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting temperature...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtapp: Assign dfluxes...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting temperature...
INFO - chtapp: And, done!
INFO - chtadapter: Restore to previous state...
INFO - chtadapter: Restore finsihed.
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtapp: Assign dfluxes.

...

INFO - chtapp: Assign dfluxes...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting temperature...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!
INFO - chtadapter: Backup current state...
INFO - chtadapter: Backup finished.


step=498, pc_iterations=0.


INFO - chtapp: Assign dfluxes...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Try to set the time size to be 0.050000.
INFO - nlgeomsolver: Begin to perform the first stage of increment
INFO - nlgeomsolver: Increment has converged.
INFO - chtapp: Extracting temperature...
INFO - chtapp: And, done!
INFO - nlgeomsolver: Post processing from a converged increment...
INFO - nlgeomsolver: And, done!


step=499, pc_iterations=0.


INFO - nlgeomsolver: Shutting down the solver...
INFO - mastersolver: Cleaning the leftover steps from step 1.
INFO - mastersolver: And, done!
INFO - mastersolver: Shutting down the solver...


In [9]:
theta=(solverF.get_temperature_interface(3,0,1)[0:100]-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFDSN_avg%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.79606531 0.80056791 0.80518476 0.80985771 0.81453247 0.81915993
 0.82369927 0.82811947 0.83239878 0.83652346 0.8404862  0.84428415
 0.84791736 0.85138821 0.85470199 0.85786755 0.86089742 0.86380622
 0.86660772 0.8693129  0.87193007 0.87446544 0.87692367 0.87930812
 0.88162086 0.88386298 0.8860353  0.88813908 0.8901764  0.89215026
 0.89406424 0.89592205 0.89772719 0.89948253 0.9011902  0.90285173
 0.90446823 0.90604052 0.90756933 0.90905539 0.91049964 0.91190338
 0.91326828 0.91459633 0.91588966 0.91715026 0.91837984 0.91957964
 0.92075035 0.92189206 0.92300449 0.92408722 0.92513988 0.92616236
 0.92715487 0.92811802 0.92905271 0.92996003 0.93084106 0.93169679
 0.93252792 0.93333488 0.93411778 0.93487648 0.93561073 0.93632016
 0.93700436 0.93766286 0.93829524 0.93890111 0.9394801  0.94003182
 0.94055588 0.94105195 0.94151972 0.9419589  0.94236897 0.94274916
 0.94309832 0.94341518 0.94369847 0.94394699 0.94415939 0.94433379
 0.94446731 0.94455602 0.94459547 0.94458129 0.94450957 0.9443